# Mount Google Drive

In [ ]:
from google.colab import drive
import os, sys
drive.mount('/content/drive')

# Install Libraries

In [ ]:
# Install libraries
!apt install python3-rtree
!pip install geopandas
!pip install osmnx
!pip install contextily
!pip install rasterstats
!pip install awscli
!pip install mapclassify
!pip install hvplot
!pip install symfit
!pip install mpld3
!pip install lmfit

import os, sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox
ox.config(use_cache=True, log_console=True)
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
import contextily as ctx
import mapclassify
import contextily as ctx
import mpld3
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 100000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# Install AWS to download datasets from SafeGraphs

In [ ]:
!aws configure --profile safegraphws

# Download datasets from SafeGraphs

In [ ]:
!aws s3 sync s3://sg-c19-response/social-distancing/v2/ "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/" --profile safegraphws --exclude "2019/*" --exclude "2020/*" --endpoint https://s3.wasabisys.com


# Loading El Paso CGB information

In [ ]:
# Loading all the Texas CGB
CGB = gpd.read_file(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/tl_2019_48_bg.shp"
)
CGB.plot()
# Extracting the El Paso CGB only
ELP_CGB = CGB.loc[(CGB["STATEFP"] == "48") & (CGB["COUNTYFP"] == "141")]
print(ELP_CGB.head())
print(ELP_CGB.shape)
# Saving the El Paso CGB information
ELP_CGB.to_file(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB.shp"
)
ELP_CGB.plot()


# Extracting population using ELP CGB

In [ ]:
from rasterstats import zonal_stats

stats = zonal_stats(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB.shp",
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/populationDensity/population_usa28_-110_2019-07-01.tif",
)
stats[0].keys()
population = [f["count"] for f in stats]

ELP = gpd.read_file(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB.shp"
)
ELP["population"] = population
ELP.to_file(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_population.shp"
)


Creating the population matrix (population - mobility)
---

In [ ]:
# Function to retrieve the index of a CGB
def getIndexFromELPCGBdictionary(df, cgb):
    value = df["index"].loc[df["CGB"] == cgb]
    return value.values[0]


dateRange = (
    pd.date_range("2020-01", "2020-10-25", freq="D").strftime("%Y/%m/%d").tolist()
)

# Loading the El Paso population of each CGB
ELPCGB_population = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_ZIP_DEMOGRAPHIC.pkl"
)[["GEOID", "B01001e1"]]

# Loading the ELP CGBs dictionary
ELP_CGB_dictionary = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_dictionary.pkl"
)

# Loading the Origin-Destination matrix
ODmatrix = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix.npy"
)

# Declaring an empty matrix in which the final population (CGB population - mobility) will be stored.
pop = np.zeros((len(dateRange), ELP_CGB_dictionary.shape[0]))
pop = pop.astype(np.float64)
print(pop.shape)

# The Z axis. Each timestep represent a day
for z in range(len(dateRange)):
    # Getting the CGBs and their population
    for cgbpop in ELPCGB_population.values:
        # Searching the CGB in the dictionary
        if cgbpop[0] in ELP_CGB_dictionary["CGB"].values:
            # Retrieving the CGB's index
            index = getIndexFromELPCGBdictionary(ELP_CGB_dictionary, str(cgbpop[0]))
            # Computing the sum of a row. Each row represents a CGB
            ODmatrixSumRow = np.sum(ODmatrix[z][index])
            # Retrieving the mobility in the same CGB.
            ODmatrixSelfCGB = ODmatrix[z][index][index]
            # print(str(cgbpop[1]) +" - "+ str(ODmatrixSumRow) +" - "+str(ODmatrixSelfCGB))
            # Computing the total population in each CGB. CGB population - (mobility going out CGB - mobility inside CGB)
            pop[z][index] = cgbpop[1]  # - ((ODmatrixSumRow)-(ODmatrixSelfCGB))

# Replacing negatives values with zero. This occurs because the number of devices from SafeGraph is not the same number
# of population in each CGB. The SafeGraph's data is only a fraction of the population
pop[pop <= 0] = 1
np.save(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/PopulationMatrixACS2012_2016",
    pop,
)


# Mapping ELP CGB to ZIP codes

In [ ]:
# https://www.huduser.gov/portal/datasets/usps_crosswalk.html
tract_zip = pd.read_excel(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/TRACT_ZIP_092020.xlsx"
)

elpcgb = gpd.read_file(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB.shp"
)

elpcgb["ZIP"] = ""
elpcgb["ZIP_RES_RATIO"] = ""

cgbs = elpcgb["GEOID"].values

for cgb in cgbs:
    zip = tract_zip.loc[tract_zip["TRACT"] == int(cgb[:-1])]
    max = zip[zip.RES_RATIO == zip.RES_RATIO.max()]
    elpcgb.loc[elpcgb["GEOID"] == str(cgb), ["ZIP"]] = max["ZIP"].values[0]
    elpcgb.loc[elpcgb["GEOID"] == str(cgb), ["ZIP_RES_RATIO"]] = max[
        "RES_RATIO"
    ].values[0]

elpcgb = elpcgb.sort_values(by="ZIP", ascending=True)
allZIPS1 = elpcgb.ZIP.unique()

elpcgb = elpcgb.sort_values(by="ZIP", ascending=True)
elpcgb.to_file(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_508cbgs.shp"
)
allZIPS1 = elpcgb.ZIP.unique()

allZIPS = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/allZIPS.npy",
    allow_pickle=True,
)


# [DS] Adding Demographic data from Open Census Data

Creating SHP file with all the Demographic Variables
---

In [ ]:
import tarfile
import pandas as pd

file = "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/OpenCensusData/safegraph_open_census_data.tar.gz"

# List that contains the demographic tables names
tables = [
    "b00",
    "b01",
    "b02",
    "b03",
    "b07",
    "b08",
    "b09",
    "b11",
    "b12",
    "b14",
    "b15",
    "b19",
    "b20",
    "b21",
    "b22",
    "b23",
    "b25",
    "b27",
    "b99",
    "c16",
    "c17",
    "c24",
]
# Getting the ELP Census Block Groups
cgbs = elpcgb["GEOID"].values

# Iterating the demographic tables list
for table in tables:
    print(table)
    # Loading one table at a time
    tbl = pd.read_csv(
        "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/OpenCensusData/safegraph_open_census_data/data/cbg_"
        + table
        + ".csv"
    )

    # Filtering the columns that contain only the 'e' character, e means 'estimated' value.
    tbl = tbl.filter(like="e", axis=1)

    # Creating new columns in our dataframe using the columns extracted in the previous line
    columns = tbl.columns.values
    # Removing first element of the array (census_block_group)
    columns = np.delete(columns, 0)
    # print(columns)
    elpcgb = pd.concat([elpcgb, pd.DataFrame(columns=columns)])

    # Iterating the ELP Census Block Groups
    for cgb in cgbs:
        # Getting the rows for each Census Block Groups. These rows contain the columns values
        val = tbl.loc[tbl["census_block_group"] == int(cgb)]
        # print(val.shape)
        if not val.empty:
            # Removing first value of the array (census_block_group)
            val = np.delete(val.values[0], 0)
            # Assigning the values from the Demographic tables to our dataframe
            elpcgb.loc[elpcgb["GEOID"] == str(cgb), columns] = val

elpcgb.to_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_ZIP_DEMOGRAPHIC.pkl"
)


Creating SHP file with all the Demographic Variables (PERCENTAGE)
---

In [ ]:
import tarfile
import pandas as pd

file = "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/OpenCensusData/safegraph_open_census_data.tar.gz"

ELPCGB_ZIP_DEMOGRAPHIC = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_ZIP_DEMOGRAPHIC.pkl"
)

columns = ELPCGB_ZIP_DEMOGRAPHIC.columns.values
columns = columns[15 : len(columns)]

for c in columns:
    if c != "B01001e1":
        ELPCGB_ZIP_DEMOGRAPHIC[c] = (
            ELPCGB_ZIP_DEMOGRAPHIC[c] * 100
        ) / ELPCGB_ZIP_DEMOGRAPHIC["B01001e1"]

ELPCGB_ZIP_DEMOGRAPHIC.to_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_ZIP_DEMOGRAPHIC_PERCENTAGE.pkl"
)


Safegraph Open Census Field Description
---

In [ ]:
fieldDescription = pd.read_csv(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/OpenCensusData/safegraph_open_census_data/cbg_field_descriptions.csv"
)
print(fieldDescription.head())
print(fieldDescription[["table_id"]].head())

t = fieldDescription[fieldDescription["table_id"].str.contains("e")]
print(t.shape)
print(t.head())
t.to_csv(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/OpenCensusData/safegraph_open_census_data/cbg_field_descriptions_estimates.csv"
)


# [DS] Social Distance Metrics - Dataset

Extracting ELP's CGB
---
TIGER: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html

According to the Topologically Integrated Geographic Encoding and Referencing (TIGER), El Paso county has 513 Census Groups Blocks (CGB). These CGBs were extracted from TIGER, version 2019. All CGB codes are composed by the following rules: two digits for the state, three digits for the county, six digits for the census track, and the last four digits identifies the census block. In our work the digits 48 identifies the Texas state and the digits 141 identifies El Paso county.

In [ ]:
# Loading all the Texas CGB
CGB = gpd.read_file(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/tl_2019_48_bg.shp"
)
CGB.plot()
# Extracting the El Paso CGB only
ELP_CGB = CGB.loc[(CGB["STATEFP"] == "48") & (CGB["COUNTYFP"] == "141")]
print(ELP_CGB.head())
print(ELP_CGB.shape)
# Saving the El Paso CGB information
ELP_CGB.to_file(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB.shp"
)
ELP_CGB.plot()


Creating the ELP CGBs dictionary.
---
This will be used to create the OD matrix. I need it because the CGBs are in long format and the matrices should have rows and columns starting from 0.

elpcgb.to_pickle('/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_dictionary.pkl')



In [ ]:
ELP_CGB = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_ZIP_DEMOGRAPHIC.pkl"
)
ELP_CGB.plot()
elpcgb = ELP_CGB["GEOID"]

# Sorting the CGBs in ascending order
elpcgb = elpcgb.sort_values()

# Reseting the index
elpcgb = elpcgb.reset_index()

# Resetting the index again to convert it into a column
elpcgb = elpcgb.reset_index()

# Removing the original index column
elpcgb.drop("index", inplace=True, axis=1)

# Renaming the new columns
elpcgb.rename(columns={"level_0": "index", "GEOID": "CGB"}, inplace=True)

# Saving the dataframe. This is our new dictionary. This will be used to create
# the OD matrix. I need it because the CGBs are in long format and the matrices
# should have rows and columns starting from 0.
# elpcgb.to_pickle('/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_dictionary.pkl')

# print(elpcgb.head())

print(elpcgb.shape)
print(elpcgb.loc[elpcgb["CGB"] == "481410103261"])


Creating the OD matrix
---
The OD matrix is created using the Social Distancing Metrics dataset and the ELP CGBs dictionary.

In [ ]:
import json
import ast
import os.path


def getIndexFromELPCGBdictionary(df, cgb):
    value = df["index"].loc[df["CGB"] == cgb]
    return value.values[0]


# Generating the date range list based on the number of days available in the
dateRange = (
    pd.date_range("2021-01-01", "2021-04-16", freq="D").strftime("%Y/%m/%d").tolist()
)

# Loading the ELP CGBs dictionary
ELP_CGB_dictionary = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_dictionary.pkl"
)

# Declaring an empty OD matrix
ODmatrix = np.zeros(
    (len(dateRange), ELP_CGB_dictionary.shape[0], ELP_CGB_dictionary.shape[0])
)
ODmatrix = ODmatrix.astype(np.float64)

zIndex = 0

for date in dateRange:
    # print(date)
    print(zIndex)
    dateSplit = date.split("/")
    file = (
        "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/"
        + dateSplit[0]
        + "/"
        + dateSplit[0]
        + "-"
        + dateSplit[1]
        + "-"
        + dateSplit[2]
        + "-social-distancing.pkl"
    )
    if os.path.isfile(file):
        # Reading social distancing metrics file
        socialDistancing = pd.read_pickle(file)

        # REMOVING all CBGs from the Destination that not are in the CBG dictionary
        socialDistancing = socialDistancing[
            socialDistancing["origin_census_block_group"].isin(
                ELP_CGB_dictionary["CGB"].values
            )
        ]

        # Updating the index in order to create the 'origin' column in the new frame.
        socialDistancing.set_index("origin_census_block_group", inplace=True, drop=True)

        # Converting the 'destination_cbgs' column from the string representation of a dictionary to a real dictionary
        socialDistancing["destination_cbgs"] = socialDistancing[
            "destination_cbgs"
        ].apply(ast.literal_eval)

        # Spliting the dictionaries to access all the elements.
        s = socialDistancing["destination_cbgs"].apply(pd.Series).stack().explode()

        # Creating the Origin-Destination list
        ODlist = s.reset_index()

        # REMOVING all CBGs from the Destination that not are in the CBG dictionary
        ODlist = ODlist[
            ODlist["origin_census_block_group"].isin(ELP_CGB_dictionary["CGB"].values)
        ]
        ODlist = ODlist[ODlist["level_1"].isin(ELP_CGB_dictionary["CGB"].values)]

        # Renaming the columns
        ODlist.columns = ["origin", "destination", "numberDevices"]

        ODlistValues = ODlist.values

        # l = ODlist.head()
        # l = l.values
        for v in ODlistValues:
            if v[1] in ELP_CGB_dictionary["CGB"].values:
                row = getIndexFromELPCGBdictionary(ELP_CGB_dictionary, str(v[0]))
                col = getIndexFromELPCGBdictionary(ELP_CGB_dictionary, str(v[1]))
                ODmatrix[zIndex][row][col] = float(v[2])

    zIndex = zIndex + 1
np.save(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix_daily_2021",
    ODmatrix,
)


Create OD-matrix 2019
---

In [ ]:
# Date Range
# dateRange = pd.date_range('2020-01','2020-10-25', freq='D').strftime("%Y/%m/%d").tolist()
import json
import ast
import calendar
import os.path
import warnings

warnings.filterwarnings("ignore")


def getIndexFromELPCGBdictionary(df, cgb):
    value = df["index"].loc[df["CGB"] == cgb]
    return value.values[0]


# Loading the ELP CGBs dictionary
ELP_CGB_dictionary = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_dictionary.pkl"
)

# Declaring an empty OD matrix
# 84 = 7(SMTWTFS) * 12 (months)
ODmatrix = np.zeros((84, ELP_CGB_dictionary.shape[0], ELP_CGB_dictionary.shape[0]))
ODmatrix = ODmatrix.astype(np.float64)

zIndex = 0

year = 2019

days = ["W-SUN", "W-MON", "W-TUE", "W-WED", "W-THU", "W-FRI", "W-SAT"]

for month in range(1, 13):
    # month = 12
    print(month)

    # Get total number of days for a given month
    monthTotalDays = calendar.monthrange(year, month)[1]

    # First day of each month
    startDate = str(month) + "/01/" + str(year)
    # startDate = str(month)+'/01/2020'

    # Last day of each month
    endDate = str(month) + "/" + str(monthTotalDays) + "/" + str(year)
    # endDate = str(month)+'/'+str(monthTotalDays)+'/2020'
    for d in days:
        print(d)
        # Get all Sun - Sat of each month
        numSMTWTFS = (
            pd.date_range(start=str(startDate), end=str(endDate), freq=d)
            .strftime("%Y/%m/%d")
            .tolist()
        )
        zIndexMonth = 0
        ODmatrixMonth = np.zeros(
            (len(numSMTWTFS), ELP_CGB_dictionary.shape[0], ELP_CGB_dictionary.shape[0])
        )
        ODmatrixMonth = ODmatrixMonth.astype(np.float64)
        for date in numSMTWTFS:
            # I NEED TO count ALL SUN, MON, ETC inside of this loop.
            # print(zIndex)
            dateSplit = date.split("/")
            file = (
                "/content/drive/MyDrive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/"
                + dateSplit[0]
                + "/"
                + dateSplit[0]
                + "-"
                + dateSplit[1]
                + "-"
                + dateSplit[2]
                + "-social-distancing.pkl"
            )
            if os.path.isfile(file):
                # # Reading social distancing metrics file
                socialDistancing = pd.read_pickle(file)

                # REMOVING all CBGs from the Destination that not are in the CBG dictionary
                socialDistancing = socialDistancing[
                    socialDistancing["origin_census_block_group"].isin(
                        ELP_CGB_dictionary["CGB"].values
                    )
                ]

                # # Updating the index in order to create the 'origin' column in the new frame.
                socialDistancing.set_index(
                    "origin_census_block_group", inplace=True, drop=True
                )

                # # Converting the 'destination_cbgs' column from the string representation of a dictionary to a real dictionary
                socialDistancing["destination_cbgs"] = socialDistancing[
                    "destination_cbgs"
                ].apply(ast.literal_eval)

                # # Spliting the dictionaries to access all the elements.
                s = (
                    socialDistancing["destination_cbgs"]
                    .apply(pd.Series)
                    .stack()
                    .explode()
                )

                # # Creating the Origin-Destination list
                ODlist = s.reset_index()

                # # REMOVING all CBGs from the Destination that not are in the CBG dictionary
                ODlist = ODlist[
                    ODlist["origin_census_block_group"].isin(
                        ELP_CGB_dictionary["CGB"].values
                    )
                ]
                ODlist = ODlist[
                    ODlist["level_1"].isin(ELP_CGB_dictionary["CGB"].values)
                ]

                # #Renaming the columns
                ODlist.columns = ["origin", "destination", "numberDevices"]

                ODlistValues = ODlist.values

                # # l = ODlist.head()
                # # l = l.values
                for v in ODlistValues:
                    if v[1] in ELP_CGB_dictionary["CGB"].values:
                        row = getIndexFromELPCGBdictionary(
                            ELP_CGB_dictionary, str(v[0])
                        )
                        col = getIndexFromELPCGBdictionary(
                            ELP_CGB_dictionary, str(v[1])
                        )
                        ODmatrixMonth[zIndexMonth][row][col] = float(v[2])

                zIndexMonth = zIndexMonth + 1

        for i in range(0, ODmatrixMonth.shape[0], 1):
            np.fill_diagonal(ODmatrixMonth[i], 0)

        ODmatrix[zIndex] = np.round(np.nanmedian(ODmatrixMonth, axis=0), 0)
        zIndex = zIndex + 1

np.save(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrixBaseline2019_nanmedian",
    ODmatrix,
)


Remove main diagonal 2019-2020
---

In [ ]:
np.set_printoptions(threshold=sys.maxsize)

ODmatrixBaseline2019 = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrixBaseline2019.npy"
)
ODmatrixBaseline2020 = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrixBaseline2020.npy"
)

inflowMatrix2019 = np.zeros((7, 12)).astype(np.float64)
outflowMatrix2019 = np.zeros((7, 12)).astype(np.float64)

inflowMatrix2020 = np.zeros((7, 12)).astype(np.float64)
outflowMatrix2020 = np.zeros((7, 12)).astype(np.float64)

for i in range(0, ODmatrixBaseline2019.shape[0], 1):
    np.fill_diagonal(ODmatrixBaseline2019[i], 0)
    np.fill_diagonal(ODmatrixBaseline2020[i], 0)

sun = list(range(0, 84, 7))
mon = list(range(1, 84, 7))
tue = list(range(2, 84, 7))
wed = list(range(3, 84, 7))
thu = list(range(4, 84, 7))
fri = list(range(5, 84, 7))
sat = list(range(6, 84, 7))

days = list(range(0, 7, 1))
for day in days:
    d2019 = list(range(day, 84, 7))
    current = ODmatrixBaseline2019[d2019]
    print(current.shape)
    for month in range(0, 12, 1):

        # outflow
        sumRows = np.sum(current[month].sum(axis=1))
        outflowMatrix2019[day][month] = sumRows

        # inflow
        sumRows = np.sum(current[month].sum(axis=0))
        inflowMatrix2019[day][month] = sumRows

for day in days:
    d2019 = list(range(day, 84, 7))
    current = ODmatrixBaseline2020[d2019]
    print(current.shape)
    for month in range(0, 12, 1):

        # outflow
        sumRows = np.sum(current[month].sum(axis=1))
        outflowMatrix2020[day][month] = sumRows

        # inflow
        sumRows = np.sum(current[month].sum(axis=0))
        inflowMatrix2020[day][month] = sumRows


Load COVID-19 data
---

In [ ]:
dateSave = "_23_04_2021"
# Reading the ZIP codes
allZIPS = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/allZIPS.npy",
    allow_pickle=True,
)
dfPositives = pd.DataFrame()
for zip in allZIPS:
    # Read the COVID-19 data for each ZIP
    df = pd.read_pickle(
        "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/"
        + str(int(zip))
        + dateSave
        + "_datesFixed.pkl"
    )
    dfPositives = pd.concat([dfPositives, df], ignore_index=True)
# ===========================================================


Create daily-yearly days COVID-19
---

In [ ]:
import calendar

year = 2020

days = ["W-SUN", "W-MON", "W-TUE", "W-WED", "W-THU", "W-FRI", "W-SAT"]

dfCOVID = pd.DataFrame(columns=days)

for month in range(1, 13):
    # month = 12
    print(month)

    # Get total number of days for a given month
    monthTotalDays = calendar.monthrange(year, month)[1]

    # First day of each month
    # startDate = str(month)+'/01/2019'
    startDate = str(month) + "/01/2020"

    # Last day of each month
    # endDate = str(month)+'/'+str(monthTotalDays)+'/2019'
    weekDays = []
    endDate = str(month) + "/" + str(monthTotalDays) + "/2020"
    for d in days:
        # print(d)
        # Get all Sun - Sat of each month
        numSMTWTFS = (
            pd.date_range(start=str(startDate), end=str(endDate), freq=d)
            .strftime("%Y/%m/%d")
            .tolist()
        )
        sel = dfPositives[dfPositives["date"].isin(numSMTWTFS)]
        weekDays.append(sel["positives"].sum())
    print(weekDays)
    dfCOVID = pd.concat(
        [dfCOVID, pd.DataFrame([weekDays], columns=dfCOVID.columns)], ignore_index=True
    )

print(dfCOVID.values.T[0])


Plot daily mobility and COVID-19
---

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

viz_test = plt

viz_test.figure(figsize=(20, 10))

SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 60

# Normalize data
dfCOVIDNorm = normalize(dfCOVID.values.T)
outflowMatrix2019Norm = normalize(outflowMatrix2019)
outflowMatrix2020Norm = normalize(outflowMatrix2020)


viz_test.rc("font", size=SMALL_SIZE)  # controls default text sizes
viz_test.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
viz_test.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
viz_test.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
viz_test.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
x_pos = [i for i, _ in enumerate(outflowMatrix2019[0])]

colors = ["blue", "green", "red", "blueviolet", "magenta", "darkorange", "black"]
days = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
for day in range(0, 7, 1):
    viz_test.plot(
        x_pos,
        outflowMatrix2019Norm[day],
        color=colors[day],
        label=days[day] + " - 2019",
        marker="o",
    )
    viz_test.plot(
        x_pos,
        outflowMatrix2020Norm[day],
        color=colors[day],
        label=days[day] + " - 2020",
        marker="o",
        linestyle="--",
    )
    viz_test.plot(
        x_pos,
        dfCOVIDNorm[day],
        color=colors[day],
        label=days[day] + " - COVID-19 2020",
        marker="o",
        linestyle="dotted",
    )

viz_test.title("El Paso County, Texas, USA - 2019 Community Mobility Patterns")
viz_test.xlabel("\n2019")
viz_test.ylabel("Mobility (Mobile Devices)")
viz_test.legend(loc="center left", bbox_to_anchor=(1, 0.5))
viz_test.grid(axis="x", color="0.75")
viz_test.xticks(
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    [
        "Jan",
        "Feb",
        "Mar",
        "Apr",
        "May",
        "Jun",
        "Jul",
        "Aug",
        "Sep",
        "Oct",
        "Nov",
        "Dec",
    ],
)
viz_test.show()


Extract ELP CBGs from SafeGraph
---

In [ ]:
import os.path

dateRange = (
    pd.date_range("2021-01-01", "2021-12-31", freq="D").strftime("%Y/%m/%d").tolist()
)
# Loading the ELP CGBs dictionary
ELP_CGB_dictionary = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_dictionary.pkl"
)

for date in dateRange:
    print(date)
    dateSplit = date.split("/")
    file = (
        "/content/drive/MyDrive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/"
        + dateSplit[0]
        + "/"
        + dateSplit[1]
        + "/"
        + dateSplit[2]
        + "/"
        + dateSplit[0]
        + "-"
        + dateSplit[1]
        + "-"
        + dateSplit[2]
        + "-social-distancing.csv.gz"
    )
    if os.path.isfile(file):
        socialDistancing = pd.read_csv(file, compression="gzip")
        socialDistancing = socialDistancing[
            socialDistancing["origin_census_block_group"].isin(
                ELP_CGB_dictionary["CGB"].values
            )
        ]
        socialDistancing.to_pickle(
            "/content/drive/MyDrive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/"
            + dateSplit[0]
            + "/"
            + dateSplit[0]
            + "-"
            + dateSplit[1]
            + "-"
            + dateSplit[2]
            + "-social-distancing.pkl"
        )
    else:
        print("File not exist")


Create OD-Matrix using Baseline
---

In [ ]:
# createODMatrix(beginDate, endDate, option): This function takes three arguments: beginDate, endDate, and option. 
# It loads an OD matrix from a file using the np.load() function, depending on the value of option it loads one of 
# three different files. It then creates a date range using the pd.date_range() function, which takes the beginDate and 
# endDate arguments as inputs. The function then splits the first date in the date range, and creates a 3D matrix of zeros 
# with the dimensions of the date range, number of rows, and number of columns. It then loops through the date range, 
# and for each date, it gets the day of the week, and uses it to index the loaded OD matrix and fill the corresponding 
# slice of the 3D matrix. Finally, it loops through the 3D matrix and fills the diagonal of each slice with zero.

import datetime
from datetime import date


def createODMatrix(beginDate, endDate, option):
    if option == "mean":
        ODmatrixBaseline2019 = np.load(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrixBaseline2019.npy"
        )
    elif option == "nanmean":
        ODmatrixBaseline2019 = np.load(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrixBaseline2019_nanmean.npy"
        )
    elif option == "nanmedian":
        ODmatrixBaseline2019 = np.load(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrixBaseline2019_nanmedian.npy"
        )

    dateRange = (
        pd.date_range(beginDate, endDate, freq="D").strftime("%Y/%m/%d").tolist()
    )
    dateSplit = dateRange[0].split("/")
    ODMatrixBaseLine = np.zeros((len(dateRange), 508, 508)).astype(np.float64)

    day_name = datetime.date(int(2021), int(5), int(8))
    dayNumber = day_name.strftime("%w")

    ac = 0
    for dateOD in dateRange:
        dateSplit = dateOD.split("/")
        indexBegin = (int(dateSplit[1]) * 7) - 7

        day_name = datetime.date(
            int(dateSplit[0]), int(dateSplit[1]), int(dateSplit[2])
        )
        dayNumber = day_name.strftime("%w")

        ODMatrixBaseLine[ac] = ODmatrixBaseline2019[indexBegin + int(dayNumber)]

        ac = ac + 1

    for i in range(0, ODMatrixBaseLine.shape[0], 1):
        np.fill_diagonal(ODMatrixBaseLine[i], 0)

    return ODMatrixBaseLine


def createODMatrixDayYear(beginDate, endDate):
    ODmatrixBaseline2019 = np.load(
        "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODMatrixBaseline2019_v2.npy"
    )
    dateRange = (
        pd.date_range(beginDate, endDate, freq="D").strftime("%Y/%m/%d").tolist()
    )
    dateRange.remove("2020/02/29")

    dateSplit = dateRange[0].split("/")
    ODMatrixBaseLine = np.zeros((len(dateRange), 508, 508)).astype(np.float64)

    ac = 0
    for dateOD in dateRange:
        dateSplit = dateOD.split("/")
        indexBegin = (int(dateSplit[1]) * 7) - 7

        day_name = datetime.date(
            int(dateSplit[0]), int(dateSplit[1]), int(dateSplit[2])
        )
        dayNumber = day_name.strftime("%w")
        ODMatrixBaseLine[ac] = ODmatrixBaseline2019[indexBegin + int(dayNumber)]
        ac = ac + 1

    return ODMatrixBaseLine


ODMatrixBaseLine = createODMatrixDayYear("2020-01-01", "2021-12-31")
np.save(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODMatrixBaseLine-2020-01-01_2021-12-31",
    ODMatrixBaseLine,
)


Plot Mobility baseline 2019 & mobility 2020
---

In [ ]:
# This code is used to create and plot a comparison of daily outflow (OD) matrices for the years 2019, 2020, 
# and 2021. The code first loads three pre-calculated OD matrices for each year: 'ODmatrix_daily_2019', 
# 'ODmatrix_daily_2020', and 'ODmatrix_daily_2021'. It then removes Feb 29th from the 2020 matrix, as it is a leap year, 
# and fills the diagonal of each matrix with zeroes. The code then uses the matplotlib library to create a line plot of the normalized OD matrix sums over the year 2019, 
# 2020, and 2021.

from datetime import datetime, date
import datetime
from datetime import date

# Specific date
day_of_year = date(2020, 2, 29).timetuple().tm_yday
dateRange2020 = (
    pd.date_range("2020-01-01", "2020-12-31", freq="D").strftime("%b-%d").tolist()
)
dateRange2020.remove("Feb-28")
ticks = [
    1,
    15,
    30,
    45,
    60,
    75,
    90,
    105,
    120,
    135,
    150,
    165,
    180,
    195,
    210,
    225,
    240,
    255,
    270,
    285,
    300,
    315,
    330,
    345,
    360,
]
dateRange2020Labels = [""] * len(dateRange2020)
for i in ticks:
    dateRange2020Labels[i - 1] = dateRange2020[i - 1]

ODmatrix_daily_2019 = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix_daily_2019.npy"
)
ODmatrix_daily_2020 = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix_daily_2020.npy"
)
ODmatrix_daily_2021 = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix_daily_2021.npy"
)

ODmatrix_daily_2020 = np.delete(ODmatrix_daily_2020, [day_of_year], axis=0)

for i in range(0, ODmatrix_daily_2019.shape[0], 1):
    np.fill_diagonal(ODmatrix_daily_2019[i], 0)

for i in range(0, ODmatrix_daily_2020.shape[0], 1):
    np.fill_diagonal(ODmatrix_daily_2020[i], 0)

for i in range(0, ODmatrix_daily_2021.shape[0], 1):
    np.fill_diagonal(ODmatrix_daily_2021[i], 0)

sum2019 = ODmatrix_daily_2019.sum(axis=(1, 2))
sum2020 = ODmatrix_daily_2020.sum(axis=(1, 2))
sum2021_ = ODmatrix_daily_2021.sum(axis=(1, 2))
sum2021_[sum2021_ == 0] = sum2021_.mean()


sum2021 = np.empty(sum2019.shape[0])
sum2021[:] = np.NaN
sum2021[0 : sum2021_.shape[0]] = sum2021_

sum2019[sum2019 == 0] = "nan"

import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

viz_test = plt

viz_test.figure(figsize=(20, 10))

SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 60

viz_test.rc("font", size=SMALL_SIZE)  # controls default text sizes
viz_test.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
viz_test.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
viz_test.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
viz_test.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
x_pos = [i for i, _ in enumerate(sum2020)]

colors = [
    "blue",
    "green",
    "red",
    "blueviolet",
    "magenta",
    "darkorange",
    "black",
    "silver",
    "cornflowerblue",
    "mediumpurple",
    "dimgray",
]
days = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
# for day in range(0,len(sum2020),1):
viz_test.plot(
    x_pos, sum2019, "--", color=colors[10], label="Mobility 2019 - Baseline"
)  # , marker='o')
viz_test.plot(x_pos, sum2020, color=colors[8], label="Mobility 2020")  # , marker='o')
viz_test.plot(x_pos, sum2021, color=colors[9], label="Mobility 2021")  # , marker='o')

viz_test.title("El Paso County, Texas, USA - Community Mobility Patterns")
viz_test.xlabel("\nDays")
viz_test.ylabel("Mobility (Mobile Devices)")

viz_test.legend()

viz_test.xticks(x_pos, dateRange2020Labels, rotation="vertical")
viz_test.show()


Compute avg day/year (Sun to Sat)
---

In [ ]:
import datetime
from datetime import date
import calendar

# Function to retrun all Mondays, Tuesdays, etc for a year
def allDays(ODmatrix_daily_2019, year, month, day):

    # Counter to track the index
    i = 0

    # Get total number of days for a given month
    monthTotalDays = calendar.monthrange(year, month)[1]

    # First day of each month
    startDate = str(month) + "/01/" + str(year)

    # Last day of each month
    endDate = str(month) + "/" + str(monthTotalDays) + "/" + str(year)

    # Get the 'day' of the 'year'. 'day' can be SUN, MON, TUE, WED, THU, FRI, SAT
    # days = pd.date_range(start=str(year), end=str(year+1), freq='W-'+day).strftime('%Y/%m/%d').tolist()
    days = (
        pd.date_range(start=str(startDate), end=str(endDate), freq="W-" + day)
        .strftime("%Y/%m/%d")
        .tolist()
    )
    # Create the np array that will hold the number of the days
    daysNumber = np.zeros((len(days))).astype(np.float64)
    ODMatrix = np.zeros((len(days), 508, 508)).astype(np.float64)

    # print(monthTotalDays, startDate, endDate)
    for day in days:
        # Split the day (dates)
        dateSplit = day.split("/")
        # Get the day name
        day_name = datetime.date(
            int(dateSplit[0]), int(dateSplit[1]), int(dateSplit[2])
        )
        # Get the number day
        dayNumber = int(day_name.strftime("%j")) - 1
        # Add the number day to the daysNumber array
        daysNumber[i] = dayNumber
        np.fill_diagonal(ODmatrix_daily_2019[int(dayNumber)], 0)
        ODMatrix[i] = ODmatrix_daily_2019[int(dayNumber)]
        i = i + 1
    averageDay = np.round(np.sum(ODMatrix.sum(axis=(0, 1))) / ODMatrix.shape[0], 0)
    return days, daysNumber, ODMatrix, averageDay


ODmatrix_daily_2019 = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix_daily_2019.npy"
)
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
dayWeek = ["SUN", "MON", "TUE", "WED", "THU", "FRI", "SAT"]
daysYearAvg = np.zeros((len(months) * len(dayWeek))).astype(np.float64)
ODMatrixBaseline2019 = np.zeros((len(months) * len(dayWeek), 508, 508)).astype(
    np.float64
)
i = 0
year = 2019
z = 0
for month in months:
    for day in dayWeek:
        # Counter to track the index
        j = 0

        # Get total number of days for a given month
        monthTotalDays = calendar.monthrange(year, month)[1]

        # First day of each month
        startDate = str(month) + "/01/" + str(year)

        # Last day of each month
        endDate = str(month) + "/" + str(monthTotalDays) + "/" + str(year)

        # Get the 'day' of the 'year'. 'day' can be SUN, MON, TUE, WED, THU, FRI, SAT
        # days = pd.date_range(start=str(year), end=str(year+1), freq='W-'+day).strftime('%Y/%m/%d').tolist()
        days = (
            pd.date_range(start=str(startDate), end=str(endDate), freq="W-" + day)
            .strftime("%Y/%m/%d")
            .tolist()
        )
        # Create the np array that will hold the number of the days
        daysNumber = np.zeros((len(days))).astype(np.float64)
        ODMatrix = np.zeros((len(days), 508, 508)).astype(np.float64)

        for day in days:
            # Split the day (dates)
            dateSplit = day.split("/")
            # Get the day name
            day_name = datetime.date(
                int(dateSplit[0]), int(dateSplit[1]), int(dateSplit[2])
            )
            # Get the number day
            dayNumber = int(day_name.strftime("%j")) - 1
            # Add the number day to the daysNumber array
            daysNumber[j] = dayNumber
            np.fill_diagonal(ODmatrix_daily_2019[int(dayNumber)], 0)
            ODMatrix[j] = ODmatrix_daily_2019[int(dayNumber)]
            j = j + 1
            z = z + 1
        averageDay = np.round(np.sum(ODMatrix.sum(axis=(0))) / ODMatrix.shape[0], 0)
        ODMatrixBaseline2019[i] = ODMatrix.sum(axis=(0)) / ODMatrix.shape[0]

        daysYearAvg[i] = averageDay
        i = i + 1
np.save(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODMatrixBaseline2019_v2",
    ODMatrixBaseline2019,
)


# ZIP to CBG

In [ ]:
from sklearn.metrics import mean_absolute_error

dfStats = pd.DataFrame(columns=["ZIP", "POSITIVE ESTIMATED", "POSITIVES"])

ELPCGB_ZIP_DEMOGRAPHIC = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_ZIP_DEMOGRAPHIC.pkl"
)
elp_ZIP_POP_COVID = ELPCGB_ZIP_DEMOGRAPHIC[
    ["GEOID", "INTPTLAT", "geometry", "ZIP", "ZIP_RES_RATIO", "B01001e1"]
].copy()

dates = pd.date_range("01-01-2020", "20-04-2021", freq="B")
# Adding new columns. Each new column is a day
dateSave = "_23_04_2021"
# Reading the ZIP codes
allZIPS = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/allZIPS.npy",
    allow_pickle=True,
)
for zip in allZIPS:
    print(zip)
    # Read the COVID-19 data for each ZIP
    df = pd.read_pickle(
        "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/"
        + str(int(zip))
        + dateSave
        + "_datesFixed.pkl"
    )
    # Load these columns only
    date_positives = df[["date", "positives", "recoveries", "deaths"]]

    # Change the 'date' column from datetime to str
    date_positives["date"] = date_positives["date"].astype(str)

    # Select all CBGs that belong to a ZIP
    zipSelected = elp_ZIP_POP_COVID.loc[elp_ZIP_POP_COVID["ZIP"] == zip].copy()  # GEOID
    # Change the datatype of the columns
    zipSelected["ZIP"] = zipSelected["ZIP"].astype(float)
    zipSelected["ZIP_RES_RATIO"] = zipSelected["ZIP_RES_RATIO"].astype(float)
    zipSelected["B01001e1"] = zipSelected["B01001e1"].astype(float)

    # ZIP total population
    sumPop = zipSelected["B01001e1"].sum()

    # Computing the ratios of each CBG using the total population
    ratios = zipSelected["B01001e1"] / sumPop

    for index, row in date_positives.iterrows():
        posEst = pd.DataFrame()
        # Estimate the positive cases in each CBG using the total positive cases for a given ZIP
        # Estimations without round
        posEstRaw = (row["positives"] * ratios) / (ratios.sum())
        # Estimations rounded
        posEst = np.round(posEstRaw, 0)
        # Compute the difference between the REAL values - estimations
        estDiff = row["positives"] - np.sum(posEst)
        # print('estDiff: '+str(estDiff))
        # If underestimate
        if estDiff > 0:
            # Sort in descending order the raw estimations
            top = posEstRaw.sort_values(ascending=False)
            # Get the top N = estDiff estimations and adds 1
            temp = np.round(top.head(int(estDiff)) + 1)
            # print(temp)
            # Update the posEst values with the new values
            posEst.loc[temp.index.values] = temp.values

        # If overestimate
        elif estDiff < 0:
            # Convert to positive
            estDiff = estDiff * -1
            # Sort in descending order the raw estimations
            top = posEstRaw.sort_values(ascending=False)
            # Get the top N = estDiff estimation
            top = top.head(int(estDiff))
            # Sort in ascending order the raw values to substract 1 to the less important CBG
            top = top.sort_values(ascending=True)
            # Substract 1 and round up
            temp = np.round(top - 1, 0)
            # Update the posEst values with the new values
            posEst.loc[temp.index.values] = temp.values

        for cell in posEst.index:
            elp_ZIP_POP_COVID.at[cell, row["date"]] = posEst.loc[cell]

        dfStats = dfStats.append(
            {
                "ZIP": zip,
                "POSITIVE ESTIMATED": np.sum(posEst),
                "POSITIVES": row["positives"],
            },
            ignore_index=True,
        )
        # print('-'*20)
elp_ZIP_POP_COVID.to_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/elp_ZIP_POP_COVID_V2.pkl"
)


# ELP Strong Web Scrapping

Web Scrapping
---

In [ ]:
import re
import json
import base64
import requests
from bs4 import BeautifulSoup

url = "https://www.epstrong.org/results.php"

soup = BeautifulSoup(requests.get(url).content, "html.parser")
html_data = requests.get(soup.iframe["src"]).text

d = json.loads(base64.b64decode(soup.iframe["src"].split("=")[-1]).decode("utf-8"))

tenantId = d["t"]
resourceKey = d["k"]
resolvedClusterUri = re.search(r"var resolvedClusterUri = '(.*?)'", html_data)[
    1
].replace("-redirect", "-api")
requestId = re.search(r"var requestId = '(.*?)'", html_data)[1]
activityId = re.search(r"var telemetrySessionId =  '(.*?)'", html_data)[1]

# https://wabi-us-north-central-b-api.analysis.windows.net/public/reports/686caa65-2ec0-49e7-86c6-a58220a4a3ef/modelsAndExploration?preferReadOnlySession=true
url = (
    resolvedClusterUri
    + "public/reports/"
    + resourceKey
    + "/modelsAndExploration?preferReadOnlySession=true"
)

# https://wabi-us-north-central-b-api.analysis.windows.net/public/reports/querydata?synchronous=true
query_url = resolvedClusterUri + "public/reports/querydata?synchronous=true"

# {'ActivityId': '54b68dbe-c5b1-46dc-99a8-8169388e9f2c', 'RequestId': '4f8900f0-7bc5-466b-b530-2ed8477f3638', 'X-PowerBI-ResourceKey': '686caa65-2ec0-49e7-86c6-a58220a4a3ef'}
headers = {
    "ActivityId": activityId,
    "RequestId": requestId,
    "X-PowerBI-ResourceKey": resourceKey,
}
# print(headers)

ELPCGB_ZIP_DEMOGRAPHIC = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_ZIP_DEMOGRAPHIC.pkl"
)
ELPCGB_ZIP_DEMOGRAPHIC = ELPCGB_ZIP_DEMOGRAPHIC.sort_values(by="ZIP", ascending=True)
allZIPS = ELPCGB_ZIP_DEMOGRAPHIC.ZIP.unique()
# print((allZIPS))

# Fort Bliss
# https://texas.hometownlocator.com/zip-codes/map,zipcode,79906.cfm
allZIPS = allZIPS[allZIPS != 79906.0]

# Empty ZIP? It seems that nobody lives here
# ZIP Code 79929 is contained within ZIP Code 79928 and perhaps demographic information for this ZIP Code would be of interest to
allZIPS = allZIPS[allZIPS != 79929.0]

# ZIP Code 79995 is contained within ZIP Code 79905 and perhaps demographic information for this ZIP Code would be of interest to
allZIPS = allZIPS[allZIPS != 79995.0]

# ZIP Code 79913 is contained within ZIP Code 79912 and perhaps demographic information for this ZIP Code would be of interest to
allZIPS = allZIPS[allZIPS != 79913.0]

# https://texas.hometownlocator.com/zip-codes/map,zipcode,79908.cfm
allZIPS = allZIPS[allZIPS != 79908.0]

# ZIP Code 79996 is contained within ZIP Code 79936 and perhaps demographic information for this ZIP Code
allZIPS = allZIPS[allZIPS != 79996.0]

allZIPS = np.append(allZIPS, 79911.0)
allZIPS = allZIPS.astype(int)

for zip in allZIPS:
    # print(zip)
    payload = {
        "version": "1.0.0",
        "queries": [
            {
                "Query": {
                    "Commands": [
                        {
                            "SemanticQueryDataShapeCommand": {
                                "Query": {
                                    "Version": 2,
                                    "From": [
                                        {
                                            "Name": "m",
                                            "Entity": "Measures Table",
                                            "Type": 0,
                                        },
                                        {
                                            "Name": "d",
                                            "Entity": "Dashboard Data RedCap",
                                            "Type": 0,
                                        },
                                    ],
                                    "Select": [
                                        {
                                            "Measure": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "m"}
                                                },
                                                "Property": "Cumulative Cases",
                                            },
                                            "Name": "Measures Table.Cumulative Cases",
                                        },
                                        {
                                            "Measure": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "m"}
                                                },
                                                "Property": "Cumulative Deaths",
                                            },
                                            "Name": "Measures Table.Cumulative Deaths",
                                        },
                                        {
                                            "Column": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "d"}
                                                },
                                                "Property": "Case Creation Date",
                                            },
                                            "Name": "Dashboard Data RedCap.Case Creation Date",
                                        },
                                        {
                                            "Measure": {
                                                "Expression": {
                                                    "SourceRef": {"Source": "m"}
                                                },
                                                "Property": "Cumulative Recoveries",
                                            },
                                            "Name": "Measures Table.Cumulative Recoveries",
                                        },
                                    ],
                                    "Where": [
                                        {
                                            "Condition": {
                                                "In": {
                                                    "Expressions": [
                                                        {
                                                            "Column": {
                                                                "Expression": {
                                                                    "SourceRef": {
                                                                        "Source": "d"
                                                                    }
                                                                },
                                                                "Property": "Zip Code",
                                                            }
                                                        }
                                                    ],
                                                    "Values": [
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'"
                                                                    + str(int(zip))
                                                                    + "'"  # "'79902'"
                                                                }
                                                            }
                                                        ]
                                                    ],
                                                }
                                            }
                                        },
                                        {
                                            "Condition": {
                                                "In": {
                                                    "Expressions": [
                                                        {
                                                            "Column": {
                                                                "Expression": {
                                                                    "SourceRef": {
                                                                        "Source": "d"
                                                                    }
                                                                },
                                                                "Property": "Zip Code",
                                                            }
                                                        }
                                                    ],
                                                    "Values": [
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79835'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79836'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79838'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79849'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79853'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79901'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79902'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79903'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79904'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79905'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79907'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79911'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79912'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79915'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79922'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79924'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79925'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79927'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79928'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79929'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79930'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79932'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79934'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79935'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79936'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79938'"
                                                                }
                                                            }
                                                        ],
                                                        [
                                                            {
                                                                "Literal": {
                                                                    "Value": "'79821'"
                                                                }
                                                            }
                                                        ],
                                                    ],
                                                }
                                            }
                                        },
                                    ],
                                },
                                "Binding": {
                                    "Primary": {
                                        "Groupings": [{"Projections": [0, 1, 2, 3]}]
                                    },
                                    "DataReduction": {
                                        "DataVolume": 4,
                                        "Primary": {"BinnedLineSample": {}},
                                    },
                                    "Version": 1,
                                },
                                "ExecutionMetricsKind": 1,
                            }
                        }
                    ]
                },
                "QueryId": "",
                "ApplicationContext": {
                    "DatasetId": "198d07f2-2ce6-402e-84f6-2fa0e46056b5",
                    "Sources": [{"ReportId": "2f58c014-b3b1-4d53-9169-73a6a382c791"}],
                },
            }
        ],
        "cancelQueries": [],
        "modelId": 6406266,
    }
    print(payload)
    # fasdfasdf
    # Sending the data
    section_data = requests.post(query_url, json=payload, headers=headers).json()
    # Extracting the data series (date, positives, recoveries, deaths)
    dataResponse = section_data["results"][0]["result"]["data"]["dsr"]["DS"][0]["PH"][
        0
    ]["DM0"]

    # Parsing the JSON response
    import json

    result = json.dumps(dataResponse)
    json_object = json.loads(str(result))

    # Converting the JSON response to a dataframe
    df = pd.DataFrame.from_dict(json_object, orient="columns")
    # print(df)
    del df["S"]
    # Traverse the C column to update it.
    for index in range(0, df.C.shape[0]):
        # If the length of df.C[index] == 3, this means that there's no deaths for this day
        # and I need to update the list to insert a 0 (deaths) between
        if len(df.C[index]) == 3:
            # Adding a 0 at position 2
            df.C[index].insert(2, 0)
            # Converting from list to string
            df.C[index] = " ".join(map(str, df.C[index]))
        elif len(df.C[index]) == 2:
            # Adding a 0 at position 2
            df.C[index].insert(2, 0)
            df.C[index].insert(3, 0)
            # Converting from list to string
            df.C[index] = " ".join(map(str, df.C[index]))
        else:
            # Converting from list to string
            df.C[index] = " ".join(map(str, df.C[index]))
    # Replacing NaN values with 0
    df.replace(np.nan, 0, inplace=True)
    # Splitting the column C. Originally was a single column with three values.
    df[["epoc_date", "positives", "deaths", "recoveries"]] = df.C.str.split(
        " ", expand=True
    )
    # Removing the original column C
    del df["C"]
    import datetime

    # Changing the datatype of the columns
    df["epoc_date"] = df["epoc_date"].astype(int)
    df["positives"] = df["positives"].astype(float)
    df["deaths"] = df["deaths"].astype(float)
    df["recoveries"] = df["recoveries"].astype(float)

    # Converting EPOCH time to human-readable format
    df["date"] = pd.to_datetime(df["epoc_date"], unit="ms")

    from datetime import datetime

    now = datetime.now()
    # datetime_backup = now.strftime("%d_%m_%Y_%H_%M_%S")
    datetime_backup = now.strftime("%d_%m_%Y")

    df.to_pickle(
        "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/"
        + str(int(zip))
        + "_"
        + datetime_backup
        + ".pkl"
    )


Fixing dates
---

In [ ]:
from pandas import Series


def difference(dataset, interval=1):
    lastValue = 0
    diff = list()
    diff.append(dataset[0])
    for i in range(interval, len(dataset)):
        # lastValue = dataset[i - 1]
        if dataset[i - 1] > 0:
            lastValue = dataset[i - 1]
        # value = dataset[i] - dataset[i - interval]
        value = dataset[i] - lastValue  # dataset[i - interval]
        if dataset[i] > 0:
            diff.append(value)
        elif lastValue == 0:
            diff.append(dataset[i])
        elif dataset[i] == 0:
            diff.append(dataset[i])
    return Series(diff)


# Generating the date range
dates = pd.date_range("01-01-2020", "20-04-2021", freq="D")
# Sufix for the file name
dateSave = "_23_04_2021"
# Reading the ZIP codes
allZIPS = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/allZIPS_ELP.npy",
    allow_pickle=True,
)
for zip in allZIPS:
    # print(zip)
    df = pd.read_pickle(
        "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/"
        + str(int(zip))
        + dateSave
        + ".pkl"
    )
    # print(df)
    # Transform from cumulative to daily cases
    # dif = difference(df['recoveries'],1)
    df["positives"] = difference(df["positives"], 1)
    df["recoveries"] = difference(df["recoveries"], 1)
    df["deaths"] = difference(df["deaths"], 1)

    # ====================================
    # Adding missing dates with nan values
    # ====================================

    # Reasigning the column date as the new index
    df.set_index("date", inplace=True)
    # Converting the new index to DatetimeIndex type
    df.index = pd.DatetimeIndex(df.index)
    # Adding the missing dates with nan values
    df = df.reindex(dates, fill_value=np.nan)
    # Removing the date column as index
    df.reset_index(level=0, inplace=True)
    # Renaming the new index column as date
    df.rename({"index": "date"}, axis=1, inplace=True)
    # Saving the fixed dataframe
    df.to_pickle(
        "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/"
        + str(int(zip))
        + dateSave
        + "_datesFixed.pkl"
    )

# Merging zips 79911 and 79912
zip79911 = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/79911"
    + dateSave
    + "_datesFixed.pkl"
)
zip79912 = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/79912"
    + dateSave
    + "_datesFixed.pkl"
)
zip79912["positives"] = zip79912["positives"] + zip79911["positives"]
zip79912["deaths"] = zip79912["deaths"] + zip79911["deaths"]
zip79912["recoveries"] = zip79912["recoveries"] + zip79911["recoveries"]
# print(zip79912)
zip79912.to_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/79912"
    + dateSave
    + "_datesFixed.pkl"
)


# SEIR Model

Load ELP Population data
---

In [ ]:
ELPCGB_population = gpd.read_file(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_population.shp"
)[["GEOID", "population"]]
print(ELPCGB_population["population"].sum())


Load OD matrix & ELP CGB dictionary
---

In [ ]:
def getIndexFromELPCGBdictionary(df, cgb):
    value = df["CGB"].loc[df["index"] == cgb]
    return value.values[0]


OD_matrices = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix.npy"
)
elpcgb = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_dictionary.pkl"
)

OD_matrices = np.where(OD_matrices == 0, 1, OD_matrices)


Generating Outbound - Inbound data
---





In [ ]:
def getIndexFromELPCGBdictionary(df, cgb):
    value = df["index"].loc[df["CGB"] == cgb]
    return value.values[0]


def getCBG(df, index):
    value = df["CGB"].loc[df["index"] == index]
    return value.values[0]


ELPCGB_ZIP_DEMOGRAPHIC = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_ZIP_DEMOGRAPHIC.pkl"
)
ELP_outbound = ELPCGB_ZIP_DEMOGRAPHIC[
    [
        "STATEFP",
        "COUNTYFP",
        "TRACTCE",
        "BLKGRPCE",
        "GEOID",
        "NAMELSAD",
        "MTFCC",
        "FUNCSTAT",
        "ALAND",
        "AWATER",
        "INTPTLAT",
        "INTPTLON",
        "geometry",
        "ZIP",
        "B01001e1",
    ]
].copy()
ELP_inbound = ELPCGB_ZIP_DEMOGRAPHIC[
    [
        "STATEFP",
        "COUNTYFP",
        "TRACTCE",
        "BLKGRPCE",
        "GEOID",
        "NAMELSAD",
        "MTFCC",
        "FUNCSTAT",
        "ALAND",
        "AWATER",
        "INTPTLAT",
        "INTPTLON",
        "geometry",
        "ZIP",
        "B01001e1",
    ]
].copy()

OD_matrices_copy = OD_matrices.copy()
np.fill_diagonal(OD_matrices_copy[0], 0)

outbound = OD_matrices_copy.sum(axis=2)
inbound = OD_matrices_copy.sum(axis=1)
print(outbound[0].shape)
print(outbound[0][0])

for time in range(0, outbound.shape[0]):
    ELP_outbound["time" + str(time)] = 0
    for iCBG in range(0, outbound.shape[1]):
        cgb = getCBG(elpcgb, iCBG)
        ELP_outbound.loc[ELP_outbound.GEOID == cgb, "time" + str(time)] = outbound[
            time
        ][iCBG]
        ELP_inbound.loc[ELP_inbound.GEOID == cgb, "time" + str(time)] = inbound[time][
            iCBG
        ]

ELP_outbound.to_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELP_outbound.pkl"
)
ELP_inbound.to_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELP_inbound.pkl"
)


Loading Outbound - Inbound data
---

In [ ]:
ELP_outbound = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELP_outbound.pkl"
)
ELP_inbound = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELP_inbound.pkl"
)


Creating Inbound - Outbound maps
---

In [ ]:
from datetime import datetime


def mobilityCBGgraph(df, time, figName, title):
    plt.figure()
    cmap = "Oranges"
    plt.rcParams.update({"font.size": 32})
    west, south, east, north = df.unary_union.bounds

    fig, ax = plt.subplots(figsize=(20, 20))
    fig.suptitle(title, fontsize=40)
    df.plot(ax=ax, column=time, legend=False, cmap=cmap)

    cbax = fig.add_axes([0.915, 0.175, 0.02, 0.7])

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=2000))
    # norm = plt.Normalize(vmin=min(df[time]), vmax=max(df[time])))

    sm._A = []

    # draw colormap into cbax

    fig.colorbar(sm, cax=cbax, format="%d")

    ax.set_xlim(west, east)
    ax.set_ylim(south, north)
    # ax.axis('off')
    ctx.add_basemap(ax, zoom=13, crs=4326)
    # plt.show()
    fig.savefig(
        "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/Mobility/" + figName
    )
    plt.close("all")


dateRange = (
    pd.date_range("2020-01", "2020-10-25", freq="D").strftime("%Y/%m/%d").tolist()
)
print(len(dateRange))
for i in range(0, 299):
    # for i in range(0,1):
    datetime_str = dateRange[i] + " 00:00:00"
    datetime_object = datetime.strptime(datetime_str, "%Y/%m/%d %H:%M:%S")
    date = datetime_object.strftime("%A, %B %d, %Y")
    title = "Outbound | " + str(date)
    mobilityCBGgraph(ELP_outbound, "time" + str(i), "outbound/outbound" + str(i), title)
    title = "Inbound | " + str(date)
    mobilityCBGgraph(ELP_inbound, "time" + str(i), "inbound/inbound" + str(i), title)


Loading Population Matrix
---

In [ ]:
np.set_printoptions(suppress=True, precision=3)
pop = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/PopulationMatrixACS2012_2016.npy"
)


V2.1 - ELP Strong (COVID data) + SafeGraph (Mobility data)
---

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import os, sys

np.set_printoptions(suppress=True)
mpld3.disable_notebook()

# Function to retrieve the ID of a CBG
def getIndex(df, cgb):
    value = df["index"].loc[df["CGB"] == cgb]
    return value.values[0]


# Function to retrieve the CBG from the ID
def getCBG(df, index):
    value = df["CGB"].loc[df["index"] == index]
    return value.values[0]


# ===========================================================
# Loading COVID data
# ===========================================================
# TODO: rows/columns meaning
elp_ZIP_POP_COVID = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/elp_ZIP_POP_COVID_V2.pkl"
)
elp_ZIP_POP_COVID.fillna(0, inplace=True)
# ===========================================================

# ===========================================================
# Load the Population Matrix
# ===========================================================
population = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/PopulationMatrixACS2012_2016.npy"
)
# ===========================================================

# ===========================================================
# Load the ELP CBG dictionary
# ===========================================================
ELP_CGB_dictionary = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_dictionary.pkl"
)
# ===========================================================

# ===========================================================
# Load the O-D matrix
# ===========================================================
# OD_matrices = np.load("/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix.npy")
OD_matrices = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODMatrixBaseLine-2020-01-01_2021-12-31.npy"
)
# Replace 0's with 1's (with 0's occur an error dividing by 0)
OD_matrices = np.where(OD_matrices == 0, 1, OD_matrices)
OD_matrices = OD_matrices.astype(np.float64)
# print("OD_matrices: ",OD_matrices.shape)
# ===========================================================

print("Total Population: " + str(np.sum(population[0])))


# ===========================================================
# Specify the number of infected people in each CBG
# ===========================================================
initialDate = "2020-04-12"


initialInd = []
initial = []
CBG_date = elp_ZIP_POP_COVID[["GEOID", initialDate]]
# print(CBG_date.head())
for index, row in CBG_date.iterrows():
    # Initial CGB indices where people are infected
    initialInd.append(getIndex(ELP_CGB_dictionary, row["GEOID"]))
    # Number of infected people in each CGB
    initial.append(row[initialDate])

initialCases = np.zeros(CBG_date.shape[0])
initialCases[initialInd] = initial

# Max number of days that I can simulate
MAX_DAYS = 300
# Number of days for the simulation
simulationDays = []
dateRange = (
    pd.date_range("2020-01", "2020-10-25", freq="D").strftime("%Y-%m-%d").tolist()
)
# index of the initial day
nStartDay = dateRange.index(initialDate)

# ===========================================================
# Number of days of the simulations
# ===========================================================
numberDaysSimulation = MAX_DAYS - nStartDay
simulationDays.append(nStartDay)
simulationDays.append(numberDaysSimulation)

# ============================================================
# SEIR model
# ============================================================
def SEIR(
    y,
    t,
    N,
    beta,
    gamma,
    delta,
    mobilityRestriction,
    outflowS,
    outflowE,
    outflowI,
    outflowR,
    inflowS,
    inflowE,
    inflowI,
    inflowR,
):
    S, E, I, R = y

    rS = S
    rE = E
    rI = I
    rR = R

    dSdt = -beta * rI * (rS / N)
    dEdt = beta * rI * (rS / N) - (delta * rE)
    dIdt = (delta * rE) - (gamma * rI)
    dRdt = gamma * rI

    return dSdt, dEdt, dIdt, dRdt


def SEIRmobility(
    y,
    t,
    N,
    beta,
    gamma,
    delta,
    mobilityRestriction,
    outflowS,
    outflowE,
    outflowI,
    outflowR,
    inflowS,
    inflowE,
    inflowI,
    inflowR,
):
    S, E, I, R = y

    rS = (S - outflowS + inflowS) * mobilityRestriction
    rE = (E - outflowE + inflowE) * mobilityRestriction
    rI = (I - outflowI + inflowI) * mobilityRestriction
    rR = (R - outflowR + inflowR) * mobilityRestriction

    dSdt = -beta * rI * (rS / N)
    dEdt = beta * rI * (rS / N) - (delta * rE)
    dIdt = (delta * rE) - (gamma * rI)
    dRdt = gamma * rI

    return dSdt, dEdt, dIdt, dRdt


# ============================================================
# Initial values of each CBG, Susceptible, Infected, and Recovered
# ============================================================

numberCBG = population.shape[1]

population = population[0]
aE = np.zeros(numberCBG, dtype=float)
aI = initialCases
aR = np.zeros(numberCBG, dtype=float)
aS = np.array(population - (aE + aI + aR), np.float)
population = aS + aE + aI + aR


# ============================================================
# Initial conditions
# ============================================================
S0, E0, I0, R0 = aS, aE, aI, aR  # initial conditions: one infected, rest susceptible
points = 300
t = np.linspace(0, points - 1, points)


# ============================================================
# Mobility
# ============================================================
# Generate random mobility data
mobility = OD_matrices  # np.random.randint(1,100,(points,len(aS),len(aS)))

# Fill main diagonal with 0s
for tm in range(0, mobility.shape[0]):
    np.fill_diagonal(mobility[tm], 0)

# Create a copy of the mobility data
mobilityNorm = mobility.copy()

# Convert the population vector from row to column in order to normalize the mobility
pop = population.reshape(len(population), 1)

# Normalize the mobility dividing it by the population of each CBG
mobilityNorm = mobilityNorm / pop

# ============================================================
# Record initial conditions
# ============================================================
# Population
N = np.zeros((len(t), len(aS))).astype(np.float64)

N[0] = aS + aE + aI + aR
populationCBGs = N.copy()

# ============================================================
# Matrix to store the results: timesteps, 4 (SEIR), N.shape (# of CGB)
# ============================================================
resultsSEIR = np.zeros((len(t), 4, N.shape[1]))
resultsSEIR[0][0] = S0
resultsSEIR[0][1] = E0
resultsSEIR[0][2] = I0
resultsSEIR[0][3] = R0

# ============================================================
# Vectors to sum the CBG and to get the regional results
# ============================================================
totalS = np.zeros((len(t))).astype(np.float64)
totalE = np.zeros((len(t))).astype(np.float64)
totalI = np.zeros((len(t))).astype(np.float64)
totalR = np.zeros((len(t))).astype(np.float64)

totalS[0] = np.sum(aS)
totalE[0] = np.sum(aE)
totalI[0] = np.sum(aI)
totalR[0] = np.sum(aR)

# ============================================================
# ELP Strong R0
# ============================================================
ELPStrong_R0 = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELPStrong_R0.pkl"
)
R0 = ELPStrong_R0["R0"].values

# print(mobilityNorm)
# ============================================================
# Running the model
# ============================================================
# for cbg in range(0,N.shape[1]):
print("*" * 40)
for timeStep in range(1, len(t)):
    for cbg in range(0, N.shape[1]):

        # ============================================================
        # Initial parameters
        # ============================================================
        D = 10.0  # infections lasts four days
        gamma = 1.0 / D
        delta = 1.0 / 5.6  # incubation period of five days
        R_0 = R0[timeStep - 1]
        beta = float(R_0) * gamma

        # popMob = N[0][cbg]
        SEIRsum = (
            resultsSEIR[timeStep - 1][0][cbg]
            + resultsSEIR[timeStep - 1][1][cbg]
            + resultsSEIR[timeStep - 1][2][cbg]
            + resultsSEIR[timeStep - 1][3][cbg]
        )

        y0 = (
            resultsSEIR[timeStep - 1][0][cbg],
            resultsSEIR[timeStep - 1][1][cbg],
            resultsSEIR[timeStep - 1][2][cbg],
            resultsSEIR[timeStep - 1][3][cbg],
        )
        tspan = [t[timeStep - 1], t[timeStep]]

        # Compute the outflow
        outflowS = np.sum(
            mobilityNorm[timeStep - 1][cbg] * resultsSEIR[timeStep - 1][0][cbg]
        )
        outflowE = np.sum(
            mobilityNorm[timeStep - 1][cbg] * resultsSEIR[timeStep - 1][1][cbg]
        )
        outflowI = np.sum(
            mobilityNorm[timeStep - 1][cbg] * resultsSEIR[timeStep - 1][2][cbg]
        )
        outflowR = np.sum(
            mobilityNorm[timeStep - 1][cbg] * resultsSEIR[timeStep - 1][3][cbg]
        )

        inflowS = np.sum(
            (mobilityNorm[timeStep - 1][:, [cbg]].T) * resultsSEIR[timeStep - 1][0][cbg]
        )
        inflowE = np.sum(
            (mobilityNorm[timeStep - 1][:, [cbg]].T) * resultsSEIR[timeStep - 1][1][cbg]
        )
        inflowI = np.sum(
            (mobilityNorm[timeStep - 1][:, [cbg]].T) * resultsSEIR[timeStep - 1][2][cbg]
        )
        inflowR = np.sum(
            (mobilityNorm[timeStep - 1][:, [cbg]].T) * resultsSEIR[timeStep - 1][3][cbg]
        )

        popMob = (
            N[0][cbg]
            - (outflowS + outflowE + outflowI + outflowR)
            + (inflowS + inflowE + inflowI + inflowR)
        )

        # Integrate the SIR ODEs
        (z, d) = odeint(
            SEIRmobility,
            y0,
            tspan,
            args=(
                popMob,
                beta,
                gamma,
                delta,
                1.0,
                outflowS,
                outflowE,
                outflowI,
                outflowR,
                inflowS,
                inflowE,
                inflowI,
                inflowR,
            ),
            full_output=1,
        )
        # print(d)
        # print('-'*10)
        # Save the new SIR results
        resultsSEIR[timeStep][0][cbg] = z[1][0]
        resultsSEIR[timeStep][1][cbg] = z[1][1]
        resultsSEIR[timeStep][2][cbg] = z[1][2]
        resultsSEIR[timeStep][3][cbg] = z[1][3]

        # Sum the results of each CBG to get the regional results
        totalS[timeStep] = totalS[timeStep] + z[1][0]
        totalE[timeStep] = totalE[timeStep] + z[1][1]
        totalI[timeStep] = totalI[timeStep] + z[1][2]
        totalR[timeStep] = totalR[timeStep] + z[1][3]

        # Updating the initial condition
        y0 = (
            resultsSEIR[timeStep][0][cbg],
            resultsSEIR[timeStep][1][cbg],
            resultsSEIR[timeStep][2][cbg],
            resultsSEIR[timeStep][3][cbg],
        )


viz_test = plt

viz_test.figure(figsize=(20, 10))

SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 60

viz_test.rc("font", size=SMALL_SIZE)  # controls default text sizes
viz_test.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
viz_test.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
viz_test.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
viz_test.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
# x_pos = [i for i, _ in enumerate(outflowMatrix2019[0])]

viz_test.plot(t, totalS, "b-", label="Susceptible")
viz_test.plot(t, totalE, "m-", label="Exposed")
viz_test.plot(t, totalI, "r-", label="Infected")
viz_test.plot(t, totalR, "g--", label="Recovered")
viz_test.plot(t, totalS + totalE + totalI + totalR, "k--", label="Total")
viz_test.ylabel("Population")
viz_test.xlabel("time")
viz_test.legend(loc="best")
viz_test.show()


In [ ]:
totalS = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/totalS.npy"
).T
totalE = np.load("/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/totalE.npy")
totalI = np.load("/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/totalI.npy")
totalR = np.load("/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/totalR.npy")


totalSm = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/totalSm.npy"
)
totalEm = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/totalEm.npy"
)
totalIm = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/totalIm.npy"
)
totalRm = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/totalRm.npy"
)

# t = totalI - totalIm

print(totalS.shape)

viz_test = plt

viz_test.figure(figsize=(20, 10))

SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 60

viz_test.rc("font", size=SMALL_SIZE)  # controls default text sizes
viz_test.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
viz_test.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
viz_test.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
viz_test.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

viz_test.plot(t, totalI, "r-", label="Infected")
viz_test.plot(t, totalIm, "r--", label="Infected M")

viz_test.plot(t, totalIm - totalI, "b--", label="Infected M")

viz_test.plot(t, totalS + totalE + totalI + totalR, "k--", label="Total")
viz_test.ylabel("Population")
viz_test.xlabel("time")
viz_test.legend(loc="best")
viz_test.show()


print(totalIm - totalI)


In [ ]:
OD_matrices = np.load(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODMatrixBaseLine-2020-01-01_2021-12-31.npy"
)


Fit model to actual data SEIRD - Dynamic R0
---

In [ ]:
import numpy as np
import random
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib import dates
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
from lmfit import minimize, Parameters, Parameter, report_fit

# !pip install numdifftools

# ******************************************************************************
# LOAD ELP STRONG R0 VALUES
# ******************************************************************************
ELPStrong_R0 = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELPStrong_R0.pkl"
)
print(ELPStrong_R0)
# dd
ELP_R0 = np.array(ELPStrong_R0["R0"].values, dtype=float)

# ******************************************************************************
# LOAD ELP STRONG COVID DATA
# ******************************************************************************
elp_ZIP_POP_COVID = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/elp_ZIP_POP_COVID_V2.pkl"
)
elp_ZIP_POP_COVID.fillna(0, inplace=True)
dfDates = elp_ZIP_POP_COVID.iloc[:, 87:-1]
elpCovidCurve = dfDates.sum()

# ===========================================================
# Get day of year using a date
# ===========================================================
from datetime import datetime, date


def getDayNumberOfTheYear(year, month, day):
    return date(year, month, day).timetuple().tm_yday - 1


# WAVE 1: 2020-04-06 - 2020-09-17
wave = 1
ELPStrong_wave = elpCovidCurve[15:180]
dateStart = "2020-04-06"
dateEnd = "2020-09-17"
index = getDayNumberOfTheYear(2020, 4, 6)
R_0_change = np.array(
    ELPStrong_R0["R0"]
    .loc[(ELPStrong_R0["date"] >= "2020-04-12") & (ELPStrong_R0["date"] <= dateEnd)]
    .values,
    dtype=float,
)
k = np.mean(np.abs(np.diff(R_0_change)))
R_0_start = float(ELPStrong_R0["R0"].loc[ELPStrong_R0["date"] == "2020-04-12"].values)
R_0_end = float(ELPStrong_R0["R0"].loc[ELPStrong_R0["date"] == dateEnd].values)
x0 = np.argmax(ELPStrong_wave)

elpCovidCurve = ELPStrong_wave

# ******************************************************************************
# LOAD ELP STRONG MORTALITY RATE VALUES
# ******************************************************************************
dfFatalityRate = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/dfFatalityRate.pkl"
)
mRate = dfFatalityRate.loc[
    (dfFatalityRate.index >= dateStart) & (dfFatalityRate.index <= dateEnd)
].values
mRate = np.array(mRate, dtype=float).mean()


# ******************************************************************************
# SEIRD MODEL
# ******************************************************************************
def SEIRmobility(y, t, N, ps):

    # N: total population
    # S(t): number of people susceptible on day t
    # E(t): number of people exposed on day t
    # I(t): number of people infected on day t
    # R(t): number of people recovered on day t
    # β (beta): expected amount of people an infected person infects per day
    # D: number of days an infected person has and can spread the disease
    # γ (gamma): the proportion of infected recovering per day (γ = 1/D)
    # R₀: the total number of people an infected person infects (R₀ = β / γ)
    # δ (delta): length of incubation period
    # α: fatality rate
    # ρ: rate at which people die (= 1/days from infected until death)

    S, E, I, R, D = y
    beta_i = betaF(t)
    gamma = ps["gamma"].value
    delta = ps["delta"].value
    alpha = ps["alpha"].value
    rho = ps["rho"].value

    beta = beta_i

    rS = S
    rE = E
    rI = I
    rR = R

    dSdt = -beta * rI * (rS / N)
    dEdt = beta * rI * (rS / N) - (delta * rE)
    dIdt = delta * rE - ((1 - alpha) * gamma * rI) - (alpha * rho * rI)
    dRdt = (1 - alpha) * gamma * rI
    dDdt = alpha * rho * rI

    return dSdt, dEdt, dIdt, dRdt, dDdt


def odesol(y, t, N, ps):
    I0 = ps["i0"].value
    y0 = S0, E0, I0, R0, D0
    x = odeint(SEIRmobility, y0, t, args=(N, ps))
    return x


def residual(ps, ts, data):
    model = pd.DataFrame(odesol(y0, t, N, ps), columns=["S", "E", "I", "R", "D"])
    return (model["I"].values - data).ravel()


bruteForceGamma = np.empty([0, 7], dtype=float)
for value in np.arange(0, 1, 1):
    # for value in np.arange(1, 9, 0.1):
    # ******************************************************************************
    # SEIR MODEL - PARAMETERS

    # β (beta): expected amount of people an infected person infects per day
    # D: number of days an infected person has and can spread the disease
    # γ (gamma): the proportion of infected recovering per day (γ = 1/D)
    # R₀: the total number of people an infected person infects (R₀ = β / γ)
    # δ (delta): length of incubation period
    # α (alpha): fatality rate
    # ρ (rho): rate at which people die (= 1/days from infected until death)

    # ******************************************************************************
    # Total population, N.
    N = 833592.0

    D = 4.0
    gamma = 1.0 / D

    beta = float(R_0_start) * gamma

    IncubationPeriod = 6.9
    delta = 1.0 / IncubationPeriod

    alpha = mRate
    rho = 1 / 9.0

    def logistic_R_0(t):
        return (R_0_start - R_0_end) / (1 + np.exp(-k * (-t + x0))) + R_0_end

    def betaF(t):
        return logistic_R_0(t) * gamma

    # The total number of people an infected person infects (R₀ = β / γ)
    # Simply explained, R0 represents the average number of people infected by one infectious individual.
    # If R0 is larger than 1, the number of infected people will likely increase exponentially, and an epidemic could ensue.
    # If R0 is less than 1, the outbreak is likely to peter out on its own.
    # R0 alone cannot definitively forecast an outbreak, but “it’s like an early warning system, in a lot of ways,
    # for the possibility of an epidemic or pandemic,”

    # ******************************************************************************
    # COMPUTING THE INITIAL VALUES
    # ******************************************************************************
    I0, R0, E0, D0 = elpCovidCurve.min(), 0, 0, 0
    # I0, R0, E0 = elpCovidCurve[0], 0, 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0
    # Initial conditions vector
    # y0 = S0, I0, R0
    y0 = S0, E0, I0, R0, D0

    # ******************************************************************************
    # CREATING THE t VECTOR
    # ******************************************************************************
    t = np.linspace(0, elpCovidCurve.shape[0] - 1, elpCovidCurve.shape[0])

    # ******************************************************************************
    # LMFIT - SET PARAMETERS
    # ******************************************************************************
    params = Parameters()
    params.add(
        "i0", elpCovidCurve[0]
    )  # , min=elpCovidCurve.min())#, max=elpCovidCurve.max())#, brute_step=300)#elpCovidCurve.max())#elpCovidCurve[10:-1].mean())#343.5)#elpCovidCurve[0:7].mean())#327.257843)#436)#, min=elpCovidCurve.min(), max=elpCovidCurve.max())
    params.add("beta_i", value=beta, min=0.01, max=beta)  # , brute_step=0.1)
    params.add("gamma", value=gamma, min=0.01, max=gamma)  # , brute_step=0.1)
    params.add("delta", value=delta, min=0.001, max=delta)  # , brute_step=0.1)
    params.add("alpha", value=alpha, min=0.001, max=alpha)  # , brute_step=0.1)
    params.add("rho", value=rho, min=0.01, max=rho)  # , brute_step=0.1)

    # ******************************************************************************
    # GET ACTUAL DATA FROM ELP STRONG
    # ******************************************************************************
    # real data
    data = elpCovidCurve.values  # dfr['totale_positivi'].values

    # ******************************************************************************
    # LMFIT - FIT MODEL AND FIND PREDICTED VALUES
    # ******************************************************************************
    # fit model and find predicted values
    result = minimize(residual, params, args=(t, data), method="leastsq")
    final = data + result.residual.reshape(data.shape)
    print("R0: ", value)
    report_fit(result)
    print("=" * 40)
    # dd
    bruteForceGamma = np.insert(
        bruteForceGamma,
        0,
        [
            value,
            result.redchi,
            result.last_internal_values[0],
            result.last_internal_values[0],
            result.last_internal_values[0],
            result.last_internal_values[0],
            result.last_internal_values[0],
        ],
        axis=0,
    )

# ******************************************************************************
# PLOT DATA AND FITTED CURVES
# ******************************************************************************
# plot data and fitted curves
plt.figure(figsize=(15, 10))
plt.plot(t, data, "-", c="silver", label="COVID-19 Data")
plt.plot(
    t,
    final,
    "--",
    linewidth=2,
    c="red",
    label="Best Fit ODE - SEIR" + " - MAX: " + str(final.max()),
)
plt.xlabel("Days")
plt.ylabel("Infected")
plt.legend(loc="best")
plt.show()


# SEIR+M Fitted


In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import os, sys

np.set_printoptions(suppress=True)
mpld3.disable_notebook()
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(suppress=True)
# ===========================================================
# Get day of year using a date
# ===========================================================
from datetime import datetime, date


def getDayNumberOfTheYear(year, month, day):
    return date(year, month, day).timetuple().tm_yday - 1


# Function to retrieve the ID of a CBG
def getIndex(df, cgb):
    value = df["index"].loc[df["CGB"] == cgb]
    return value.values[0]


# Function to retrieve the CBG from the ID
def getCBG(df, index):
    value = df["CGB"].loc[df["index"] == index]
    return value.values[0]


def SEIRmobility(
    y,
    t,
    N,
    beta,
    gamma,
    delta,
    tau,
    mobilityRestriction,
    outflowS,
    outflowE,
    outflowI,
    outflowR,
    inflowS,
    inflowE,
    inflowI,
    inflowR,
):

    # N: total population
    # S(t): number of people susceptible on day t
    # E(t): number of people exposed on day t
    # I(t): number of people infected on day t
    # R(t): number of people recovered on day t
    # β (beta): expected amount of people an infected person infects per day
    # D: number of days an infected person has and can spread the disease
    # γ (gamma): the proportion of infected recovering per day (γ = 1/D)
    # R₀: the total number of people an infected person infects (R₀ = β / γ)
    # δ (delta): length of incubation period
    # α: fatality rate
    # ρ: rate at which people die (= 1/days from infected until death)

    S, E, I, R = y

    try:
        beta_i = beta
        tau = tau
        gamma = gamma
        delta = delta
    except:
        print("--")

    beta = (beta_i * (1.1 - tau * t)) * (mobilityRestriction)

    rS = S - outflowS * mobilityRestriction + inflowS * mobilityRestriction
    rE = E - outflowE * mobilityRestriction + inflowE * mobilityRestriction
    rI = I - outflowI * mobilityRestriction + inflowI * mobilityRestriction
    rR = R - outflowR * mobilityRestriction + inflowR * mobilityRestriction

    dSdt = -beta * rI * (rS / N)
    dEdt = beta * rI * (rS / N) - (delta * rE)
    dIdt = (delta * rE) - (gamma * rI)
    dRdt = gamma * rI

    return dSdt, dEdt, dIdt, dRdt


# ===========================================================
# Load COVID cases
# ===========================================================
elp_ZIP_POP_COVID = pd.read_pickle(
    "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/elp_ZIP_POP_COVID_V2.pkl"
)
elp_ZIP_POP_COVID.fillna(0, inplace=True)
dfDates = elp_ZIP_POP_COVID.iloc[:, 87:-1]
elpCovidCurve = dfDates.sum()

# ===========================================================
# Define COVID waves
# ===========================================================
wave = 2
ELPStrong_wave = elpCovidCurve[180:280]
dateStart = "2020-09-18"
dateEnd = "2020-12-26"
indexStart = getDayNumberOfTheYear(2020, 9, 18)
elpCovidCurve = ELPStrong_wave


mobilityRestrictionExperiment = 100.0
mobRestrictionPercentage = 0.0
daysWindow = 0


mre = np.array([50.0])
dw = np.array([8])

for mobilityRestrictionExperiment in mre:
    for daysWindow in dw:
        # ===========================================================
        # MOBILITY CHANGE
        # ===========================================================
        from datetime import datetime, date
        import datetime
        from datetime import date

        day_of_year = date(2020, 2, 29).timetuple().tm_yday

        # WAVE 1: 2020-04-06 - 2020-09-17
        if wave == 1:
            firstDayNumber = date(2020, 4, 6).timetuple().tm_yday
            lastDayNumber = date(2020, 9, 17).timetuple().tm_yday + 1
        elif wave == 2:
            firstDayNumber = date(2020, 9, 18).timetuple().tm_yday
            lastDayNumber = date(2020, 12, 26).timetuple().tm_yday + 1
        elif wave == 3:
            firstDayNumber = date(2021, 1, 1).timetuple().tm_yday
            lastDayNumber = date(2021, 4, 13).timetuple().tm_yday + 1
            # print(lastDayNumber - firstDayNumber)
            # dd

        ODmatrix_daily_2019 = np.load(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix_daily_2019.npy"
        )
        ODmatrix_daily_2020 = np.load(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix_daily_2020.npy"
        )
        ODmatrix_daily_2021 = np.load(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODmatrix_daily_2021.npy"
        )

        # ===========================================================
        # Loading COVID data
        # ===========================================================
        # TODO: rows/columns meaning
        elp_ZIP_POP_COVID = pd.read_pickle(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_COVID_DATA/elp_ZIP_POP_COVID_V2.pkl"
        )
        elp_ZIP_POP_COVID.fillna(0, inplace=True)
        # print(elp_ZIP_POP_COVID.head())
        # ===========================================================

        # ===========================================================
        # Load the Population Matrix
        # ===========================================================
        population = np.load(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/PopulationMatrixACS2012_2016.npy"
        )
        # ===========================================================

        # ===========================================================
        # Load the ELP CBG dictionary
        # ===========================================================
        ELP_CGB_dictionary = pd.read_pickle(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/ELP_CGB/ELPCGB_dictionary.pkl"
        )
        # pop = np.load("/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/PopulationMatrix3.npy")
        # ===========================================================

        ODmatrix_daily_2020 = np.delete(ODmatrix_daily_2020, [day_of_year], axis=0)

        for i in range(0, ODmatrix_daily_2019.shape[0], 1):
            np.fill_diagonal(ODmatrix_daily_2019[i], 0)

        for i in range(0, ODmatrix_daily_2020.shape[0], 1):
            np.fill_diagonal(ODmatrix_daily_2020[i], 0)

        for i in range(0, ODmatrix_daily_2021.shape[0], 1):
            np.fill_diagonal(ODmatrix_daily_2021[i], 0)

        sum2019 = ODmatrix_daily_2019.sum(axis=(1, 2))
        sum2019[sum2019 == 0] = sum2019.mean()  #'nan'

        sum2020 = ODmatrix_daily_2020.sum(axis=(1, 2))

        sum2021_ = ODmatrix_daily_2021.sum(axis=(1, 2))
        sum2021_[sum2021_ == 0] = sum2021_.mean()
        sum2021 = np.empty(sum2019.shape[0])
        sum2021[:] = np.nan
        sum2021[0 : sum2021_.shape[0]] = sum2021_

        mobChange = sum2019 - sum2020
        mobChange2021 = sum2019 - sum2021
        mobChangePercentage = 1 - ((mobChange / sum2019))
        mobChangePercentage2021 = 1 - ((mobChange2021 / sum2019))

        indexI = firstDayNumber
        indexE = lastDayNumber

        mobChangePercentage = np.nan_to_num(
            ((mobChangePercentage[indexI:indexE])), nan=mobChangePercentage.mean()
        )

        # ===========================================================
        # Load the O-D matrix
        # ===========================================================
        OD_matrices = np.load(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/SafeGraphs/SocialDistancingMetrics/ODMatrixBaseLine-2020-01-01_2021-12-31.npy"
        )
        # Replace 0's with 1's (with 0's occur an error dividing by 0)
        # OD_matrices = np.where(OD_matrices==0, 1, OD_matrices)
        OD_matrices = np.where(OD_matrices == 0, 0.000000000000001, OD_matrices)
        OD_matrices = OD_matrices.astype(np.float64)
        # ===========================================================

        # ===========================================================
        # Specify the number of infected people in each CBG
        # ===========================================================
        initialDate = dateStart

        initialInd = []
        initial = []
        CBG_date = elp_ZIP_POP_COVID[["GEOID", initialDate]]
        for index, row in CBG_date.iterrows():
            # Initial CGB indices where people are infected
            initialInd.append(getIndex(ELP_CGB_dictionary, row["GEOID"]))
            # Number of infected people in each CGB
            initial.append(row[initialDate])

        initialCases = np.zeros(CBG_date.shape[0])
        initialCases[initialInd] = initial
        initialCases = (initialCases * 1) / np.sum(initialCases)

        # ============================================================
        # Initial values of each CBG, Susceptible, Infected, and Recovered
        # ============================================================

        numberCBG = population.shape[1]

        population = population[0]
        aE = np.zeros(numberCBG, dtype=float)

        if wave == 1:
            # WAVE 1
            percentage = 21.6581097 / np.sum(population)
            infectedPeopleFitted = 21.6581097
            # aI = initialCases*infectedPeopleFitted
            aI = population * percentage
        elif wave == 2:
            # WAVE 2
            percentage = 40.9305644 / np.sum(population)
            infectedPeopleFitted = 40.9305644
            aI = initialCases * infectedPeopleFitted
            # aI = population*percentage
        elif wave == 3:
            # WAVE 3
            percentage = 539.995526 / np.sum(population)
            infectedPeopleFitted = 539.995526
            # aI = initialCases*infectedPeopleFitted
            aI = population * percentage

        # print('AI: ',np.sum(aI))
        aR = np.zeros(numberCBG, dtype=float)
        aS = np.array(population - (aE + aI + aR), np.float)
        population = aS + aE + aI + aR
        # print('Total Population: ',np.sum(population))

        # ============================================================
        # Initial conditions
        # ============================================================
        S0, E0, I0, R0 = aS, aE, aI, aR
        points = len(elpCovidCurve)
        t = np.linspace(0, points - 1, points)

        # ============================================================
        # Mobility
        # ============================================================
        # Generate random mobility data
        mobility = OD_matrices

        # Fill main diagonal with 0s
        for tm in range(0, mobility.shape[0]):
            np.fill_diagonal(mobility[tm], 0)

        # Create a copy of the mobility data
        mobilityNorm = mobility.copy()

        # Convert the population vector from row to column in order to normalize the mobility
        pop = population.reshape(len(population), 1)

        # Normalize the mobility dividing it by the population of each CBG
        mobilityNorm = mobilityNorm / pop
        # ============================================================
        # Fitted parameters for each WAVE
        # ============================================================

        if wave == 1:
            # WAVE 1
            gamma = 0.42419584
            delta = 0.01898470
            beta = 2.26280976
            tau = 0.00864778

        elif wave == 2:
            # WAVE 2
            gamma = 0.37782941
            delta = 1.71527631
            beta = 0.52848914
            tau = 0.00773372

        elif wave == 3:
            # WAVE 3
            gamma = 0.23216184
            delta = 1.24411741
            beta = 0.22044555
            tau = 0.00293696

        # ============================================================
        # Record initial conditions
        # ============================================================
        # Population
        N = np.zeros((len(t), len(aS))).astype(np.float64)
        N[0] = aS + aE + aI + aR
        populationCBGs = N.copy()

        # ============================================================
        # Matrix to store the results: timesteps, 4 (SEIR), N.shape (# of CGB)
        # ============================================================
        resultsSEIR = np.zeros((len(t), 4, N.shape[1]))
        resultsSEIR[0, 0] = S0
        resultsSEIR[0, 1] = E0
        resultsSEIR[0, 2] = I0
        resultsSEIR[0, 3] = R0

        # ============================================================
        # Vectors to sum the CBG and to get the regional results
        # ============================================================
        totalS = np.zeros((len(t))).astype(np.float64)
        totalE = np.zeros((len(t))).astype(np.float64)
        totalI = np.zeros((len(t))).astype(np.float64)
        totalR = np.zeros((len(t))).astype(np.float64)

        totalS[0] = np.sum(aS)
        totalE[0] = np.sum(aE)
        totalI[0] = np.sum(aI)
        totalR[0] = np.sum(aR)

        # ============================================================
        # Running the model
        # ============================================================
        # for cbg in range(0,N.shape[1]):
        dfHotspots = pd.DataFrame(
            columns=[
                "timestep",
                "cbg",
                "casesLast7Days",
                "casesPreceding7days",
                "casesLast7DaysPercentage",
                "casesLast3Days",
                "casesPreceding3Days",
                "casesLast3DaysPercentage",
                "casesLast30Days",
                "ratio7days30days",
            ]
        )
        # dfExperiment = pd.DataFrame(columns = ['wave','timestep', 'cbg','S','E','I','R','mobRestrictionPercentage','mobilityRestrictionExperiment','daysWindow'])
        # print('*'*40)

        # ============================================================
        # MOBILITY RESTRICTIONS
        # ============================================================
        mobilityRestrictionVector = np.ones((len(t), numberCBG), dtype=float)
        if wave == 1:
            mobilityRestrictionVector[0:60, :] = mobChangePercentage[0:60, None]
        elif wave == 2:
            mobilityRestrictionVector[0:36, :] = mobChangePercentage[0:36, None]
        elif wave == 3:
            mobilityRestrictionVector[0:85, :] = mobChangePercentage2021[0:85, None]

        # dfExperiment = dfExperiment.append({'wave':wave,'timestep':0, 'cbg':cbg,'S':z[1,0],'E':z[1,1],'I':z[1,2],'R':z[1,3],'mobRestrictionPercentage':mobRestrictionPercentage,'mobilityRestrictionExperiment':mobilityRestrictionExperiment,'daysWindow':daysWindow}, ignore_index = True)
        # mobilityRestriction = 0.1
        for timeStep in range(1, len(t)):
            for cbg in range(0, N.shape[1] - 10):

                SEIRsum = (
                    resultsSEIR[timeStep - 1, 0, cbg]
                    + resultsSEIR[timeStep - 1, 1, cbg]
                    + resultsSEIR[timeStep - 1, 2, cbg]
                    + resultsSEIR[timeStep - 1, 3, cbg]
                )

                y0 = (
                    resultsSEIR[timeStep - 1, 0, cbg],
                    resultsSEIR[timeStep - 1, 1, cbg],
                    resultsSEIR[timeStep - 1, 2, cbg],
                    resultsSEIR[timeStep - 1, 3, cbg],
                )
                tspan = [t[timeStep - 1], t[timeStep]]

                # mobilityRestriction = mobilityRestrictionVector[timeStep-1,cbg]
                mobRestrictionPercentage = mobilityRestrictionVector[timeStep - 1, cbg]

                outflowS = np.sum(
                    mobilityNorm[indexStart + (timeStep - 1), cbg]
                    * resultsSEIR[timeStep - 1, 0, cbg]
                )
                outflowE = np.sum(
                    mobilityNorm[indexStart + (timeStep - 1), cbg]
                    * resultsSEIR[timeStep - 1, 1, cbg]
                )
                outflowI = np.sum(
                    mobilityNorm[indexStart + (timeStep - 1), cbg]
                    * resultsSEIR[timeStep - 1, 2, cbg]
                )
                outflowR = np.sum(
                    mobilityNorm[indexStart + (timeStep - 1), cbg]
                    * resultsSEIR[timeStep - 1, 3, cbg]
                )

                inflowS = np.sum(
                    (mobilityNorm[indexStart + (timeStep - 1), :, [cbg]].T)
                    * resultsSEIR[timeStep - 1, 0, cbg]
                )
                inflowE = np.sum(
                    (mobilityNorm[indexStart + (timeStep - 1), :, [cbg]].T)
                    * resultsSEIR[timeStep - 1, 1, cbg]
                )
                inflowI = np.sum(
                    (mobilityNorm[indexStart + (timeStep - 1), :, [cbg]].T)
                    * resultsSEIR[timeStep - 1, 2, cbg]
                )
                inflowR = np.sum(
                    (mobilityNorm[indexStart + (timeStep - 1), :, [cbg]].T)
                    * resultsSEIR[timeStep - 1, 3, cbg]
                )

                popMob = N[0][cbg]

                # Integrate the SIR ODEs
                (z, d) = odeint(
                    SEIRmobility,
                    y0,
                    tspan,
                    args=(
                        popMob,
                        beta,
                        gamma,
                        delta,
                        tau,
                        mobRestrictionPercentage,
                        outflowS,
                        outflowE,
                        outflowI,
                        outflowR,
                        inflowS,
                        inflowE,
                        inflowI,
                        inflowR,
                    ),
                    full_output=1,
                )

                resultsSEIR[timeStep, 0, cbg] = z[1, 0]
                resultsSEIR[timeStep, 1, cbg] = z[1, 1]
                resultsSEIR[timeStep, 2, cbg] = z[1, 2]
                resultsSEIR[timeStep, 3, cbg] = z[1, 3]

                # Sum the results of each CBG to get the regional results
                totalS[timeStep] = totalS[timeStep] + z[1, 0]
                totalE[timeStep] = totalE[timeStep] + z[1, 1]
                totalI[timeStep] = totalI[timeStep] + z[1, 2]
                totalR[timeStep] = totalR[timeStep] + z[1, 3]

                # 1) >100 new COVID-19 cases in the most recent 7 days,
                # 2) an increase in the most recent 7-day COVID-19 incidence over the preceding 7-day incidence,
                # 3) a decrease of <60% or an increase in the most recent 3-day COVID-19 incidence over the preceding 3-day incidence, and
                # 4) the ratio of 7-day incidence/30-day incidence exceeds 0.31.

                # In addition, hotspots must have met at least one of the following criteria:
                # 1) >60% change in the most recent 3-day COVID-19 incidence, or
                # 2) >60% change in the most recent 7-day incidence.
                if timeStep >= 29:
                    casesLast7Days = np.sum(
                        resultsSEIR[timeStep - 6 : timeStep + 1, 2, cbg]
                    )
                    casesPreceding7days = np.sum(
                        resultsSEIR[timeStep - 13 : timeStep - 6, 2, cbg]
                    )
                    casesLast7DaysPercentage = 100 - (
                        (casesPreceding7days * 100) / casesLast7Days
                    )

                    casesLast3Days = np.sum(
                        resultsSEIR[timeStep - 2 : timeStep + 1, 2, cbg]
                    )
                    casesPreceding3Days = np.sum(
                        resultsSEIR[timeStep - 5 : timeStep - 2, 2, cbg]
                    )

                    casesLast3DaysPercentage = 100 - (
                        (casesPreceding3Days * 100) / casesLast3Days
                    )

                    casesLast30Days = np.sum(
                        resultsSEIR[timeStep - 29 : timeStep + 1, 2, cbg]
                    )
                    ratio7days30days = casesLast7Days / casesLast30Days

                    if casesLast7Days > 100:
                        if casesLast7Days > casesPreceding7days:
                            if casesLast3Days > casesPreceding3Days:
                                if ratio7days30days > 0.31:
                                    if (
                                        casesLast7DaysPercentage > 60.0
                                        or casesLast3DaysPercentage > 60.0
                                    ):
                                        print(
                                            timeStep,
                                            "CBG: ",
                                            cbg,
                                            casesLast7Days,
                                            casesPreceding7days,
                                            casesLast3Days,
                                            casesPreceding3Days,
                                            casesLast3DaysPercentage,
                                            casesLast30Days,
                                            ratio7days30days,
                                        )
                                        dfHotspots = dfHotspots.append(
                                            {
                                                "timestep": timeStep,
                                                "cbg": cbg,
                                                "casesLast7Days": casesLast7Days,
                                                "casesPreceding7days": casesPreceding7days,
                                                "casesLast7DaysPercentage": casesLast7DaysPercentage,
                                                "casesLast3Days": casesLast3Days,
                                                "casesPreceding3Days": casesPreceding3Days,
                                                "casesLast3DaysPercentage": casesLast3DaysPercentage,
                                                "casesLast30Days": casesLast30Days,
                                                "ratio7days30days": ratio7days30days,
                                            },
                                            ignore_index=True,
                                        )
                                        mobRestrictionPercentage = (
                                            mobilityRestrictionExperiment / 100
                                        )  # mobilityRestrictionVector[timeStep-1,cbg] * (mobilityRestrictionExperiment/100)
                                        # if daysWindow > 0:
                                        mobilityRestrictionVector[
                                            timeStep : timeStep + daysWindow, cbg
                                        ] = mobRestrictionPercentage

        dfHotspots.to_pickle(
            "/content/drive/My Drive/Colab Notebooks/Epidemic Modeling/resultsPaper/dfHotspots_wave2t"
            + str(wave)
            + "_"
            + str(int(mobilityRestrictionExperiment))
            + "_"
            + str(daysWindow)
            + ".pkl"
        )
display(dfHotspots)
# ============================================================
# Plot the regional results
# ============================================================
viz_test = plt

viz_test.figure(figsize=(15, 10))

SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 60

viz_test.rc("font", size=SMALL_SIZE)  # controls default text sizes
viz_test.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
viz_test.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
viz_test.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
viz_test.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
viz_test.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

viz_test.plot(
    t,
    totalI,
    "r--",
    label="Infected" + "-- Max: " + str(totalI.max()) + " Min: " + str(totalI.min()),
)
viz_test.plot(t, elpCovidCurve, "-", c="silver", label="Real data")

viz_test.ylabel("Population")
viz_test.xlabel("time")
viz_test.legend(loc="best")
viz_test.show()

# ============================================================
# Plot matrix
# ============================================================
plt.figure(figsize=(25, 20))
ax = plt.gca()
im = ax.imshow(resultsSEIR[:, 2, :], cmap="hot_r")
plt.xlabel("CBG number", fontsize=30)
plt.ylabel("Time", fontsize=30)


from mpl_toolkits.axes_grid1 import make_axes_locatable

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(mappable=im, cax=cax, label="Number of infections")
